In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
from PyPDF2 import PdfReader
path='/Users/matansharon/python/chat_with_docs/data/pdf/qlora.pdf'
doc=PdfReader(path)
#get the name of the document
doc_name=path.split('/')[-1]

text=''
for page in doc.pages:
    text+=page.extract_text()



chunks=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100,length_function=len,add_start_index=True).split_text(text)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )

chroma_client = chromadb.Client()
collection=chroma_client.create_collection(name="My_pdf_collection",embedding_function= openai_ef)
metadatas=[{"page_number":i,"page_content":chunks[i],'doc_name':doc_name} for i in range(len(chunks))]
emb_list = openai_ef(chunks)
ids=["id"+str(i) for i in range(len(chunks))]
collection.add(embeddings=emb_list,documents=chunks,metadatas=metadatas,ids=ids)
res=collection.query(query_texts=["what is qlora?"],n_results=3)
res
results_with_high_score=[]
for i in range(len(res['ids'][0])):
    if res['distances'][0][i]<1.1:
        print(res['distances'][0][i])
        results_with_high_score.append(res['documents'][0][i])
results_with_high_score
client=chromadb.PersistentClient('chromadb')